In [77]:
# 

import copy
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial
import scipy.optimize
import skimage.data
import skimage.transform

# TODO: figure out a better way of doing this
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
# import register
# import utils

im = skimage.data.astronaut()
im = np.uint8(255 * skimage.transform.resize(im, (200, 250)))


def get_rbf_design_matrix(x_samp, x_data, sigma=1):
    x_s_d_dist_sq = np.power(
        scipy.spatial.distance.cdist(x_samp, x_data), 2)
    x_s_d_exp = np.exp(- x_s_d_dist_sq / sigma ** 2)
    return x_s_d_exp


# orig sampling (where full data is)
x_range = np.arange(0, im.shape[1])
y_range = np.arange(0, im.shape[0])

xx, yy = np.meshgrid(x_range, y_range)
xin, yin = xx.flatten(), yy.flatten()

# later sampling (where sparse data is)
SAMP_SPACE = 4

# resample image
x_range_samp = np.linspace(0, im.shape[1] - 1, im.shape[1] / SAMP_SPACE)
y_range_samp = np.linspace(0, im.shape[0] - 1, im.shape[0] / SAMP_SPACE)

# im_sampled 
xx_samp, yy_samp = np.meshgrid(x_range_samp, y_range_samp)
xin_samp, yin_samp = xx_samp.flatten(), yy_samp.flatten()

im_sampled = im[:, :, 0][y_range_samp.astype('uint8')][:, x_range_samp.astype('uint8')]


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


In [88]:
# reconstruct orig image based on sampling

SIG=8
x_samp = np.array([xin_samp, xin_samp]).T
H_samp = get_rbf_design_matrix(x_samp, x_samp, sigma=SIG)

y_samp = im_sampled.reshape(len(x_samp), 1)

# solve least squares
lam = .1
H_t_H = np.dot(H_samp.T, H_samp)
lam_I = lam * np.eye(H_t_H.shape[0])
H_t_H_inv = np.linalg.inv(H_t_H + lam_I)
w = np.dot(H_t_H_inv, np.dot(H_samp.T, y_samp))

# now apply to large im 
x_data_all = np.array([xin, yin]).T
H_all = get_rbf_design_matrix(x_data_all, x_samp, sigma=SIG)

y_all = np.dot(H_all, w)
im_estimated = y_all.reshape(xx.shape)

# show images
plt.figure(1)
plt.imshow(im_sampled.reshape(xx_samp.shape), cmap='gray')

plt.figure(2)
plt.imshow(im_estimated, cmap='gray')

plt.show()